In [1]:
library(readxl)
library(ggplot2)
library(caret)
library(yardstick)
library(tibble)
library(Metrics)
library(e1071)
library(ROCR)
library(Hmisc)
library(MLmetrics)
library(mccr)
library(psych)
library(irr)
library(ROCR)
library(PRROC)
library(dplyr)
library(readxl)

Warning message:
"package 'readxl' was built under R version 4.3.3"
Warning message:
"package 'ggplot2' was built under R version 4.3.3"
Warning message:
"package 'caret' was built under R version 4.3.3"
Loading required package: lattice

Warning message:
"package 'lattice' was built under R version 4.3.3"
Warning message:
"package 'yardstick' was built under R version 4.3.3"

Attaching package: 'yardstick'


The following objects are masked from 'package:caret':

    precision, recall, sensitivity, specificity


Warning message:
"package 'Metrics' was built under R version 4.3.3"

Attaching package: 'Metrics'


The following objects are masked from 'package:yardstick':

    accuracy, mae, mape, mase, precision, recall, rmse, smape


The following objects are masked from 'package:caret':

    precision, recall


Warning message:
"package 'e1071' was built under R version 4.3.3"
Warning message:
"package 'Hmisc' was built under R version 4.3.3"

Attaching package: 'Hmisc'


The followin

In [2]:
data <- read_excel("C:/Users/SADEGHI/Desktop/BinaryClassification_data.xlsx", sheet="HC_DF_SPT_Head and Neck")
head(data)
y_true  <- data$y_true
y_pred <- data$y_pred

y_true,y_pred,y_pred_proba
<dbl>,<dbl>,<dbl>
0,0,0.12
1,0,0.27
0,0,0.32
1,0,0.22
0,0,0.39
0,0,0.18


In [3]:
library(pROC)
auc(roc(y_true, y_pred))


Warning message:
"package 'pROC' was built under R version 4.3.3"
Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following object is masked from 'package:Metrics':

    auc


The following objects are masked from 'package:stats':

    cov, smooth, var


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Area under the curve: 0.4561

In [4]:
y_true <- as.numeric(y_true)
pred <- prediction(y_pred, y_true)
perf <- performance(pred, "auc")
auc_value <- perf@y.values[[1]]
formatted_auc_value <- format(auc_value, nsmall = 15)
print(formatted_auc_value)

[1] "0.456105041211424"


In [41]:
### Base R ###
accuracy <- sum(y_true == y_pred) / length(y_true)
sprintf("Base-R Accuracy:%.15f", accuracy)

conf_matrix <- table(Predicted = y_pred, Actual = y_true)
TP <- conf_matrix["1", "1"]
FP <- conf_matrix["1", "0"]
FN <- conf_matrix["0", "1"]
TN <- conf_matrix["0", "0"]
precision <- TP / (TP + FP)
sprintf("Base-R precision: %.15f", precision)
recall <- TP /(TP + FN)
sprintf("Base-R recall: %.15f", recall)
f1_score <- 2*(precision*recall)/(precision + recall)
sprintf("Base-R f1_score: %.15f", f1_score)



observed_agreement <- sum(diag(conf_matrix)) / sum(conf_matrix)

row_probs <- rowSums(conf_matrix) / sum(conf_matrix)
col_probs <- colSums(conf_matrix) / sum(conf_matrix)
expected_agreement <- sum(row_probs * col_probs)

kappa <- (observed_agreement - expected_agreement) / (1 - expected_agreement)
sprintf("Cohen's Kappa (manual): %.15f", kappa)



mcc <- (TP * TN - FP * FN) / sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print(paste("MCC (manual):", mcc))

sensitivity <- TP / (TP + FN)
specificity <- TN / (TN + FP)

balanced_accuracy = (sensitivity + specificity)/2
sprintf("Base-R Balanced Accuracy: %.15f", balanced_accuracy)


beta <- 0.5;
f_beta_score <- ((1 + beta**2) * (precision * recall)) / (beta**2 * precision + recall);
sprintf("Base-R F beta Score: %.15f", f_beta_score)



jaccard <- TP/ (TP + FP +FN)
sprintf("Base-R Jaccard Index: %.15f", jaccard)

geometric_mean_manual <- sqrt(sensitivity * specificity)
print(paste("Geometric Mean (manual):", geometric_mean_manual))

[1] "Base-R Accuracy:0.691011235955056"

[1] "Base-R precision: 0.090909090909091"

[1] "Base-R recall: 0.054054054054054"

[1] "Base-R f1_score: 0.067796610169492"

[1] "Cohen's Kappa (manual): -0.103222898354744"

[1] "MCC (manual): -0.108238414016216"


[1] "Base-R Balanced Accuracy: 0.456105041211424"

[1] "Base-R F beta Score: 0.080000000000000"

[1] "Base-R Jaccard Index: 0.035087719298246"

[1] "Geometric Mean (manual): 0.215375979032619"


In [5]:
### Caret Library ###

confMatrix <- confusionMatrix(factor(y_pred), factor(y_true))
accuracy <- confMatrix$overall['Accuracy']
sprintf("Caret Accuracy: %.15f", accuracy)
tn <- confMatrix$table[1, 1]  # True Negatives
fp <- confMatrix$table[2, 1]  # False Positives
fn <- confMatrix$table[1, 2]  # False Negatives
tp <- confMatrix$table[2, 2]  # True Positives

precision <- confMatrix$byClass['Pos Pred Value']
recall <- confMatrix$byClass['Sensitivity']
f1 <- confMatrix$byClass['F1']

sprintf("Caret precision: %.15f", precision)
sprintf("Caret recall: %.15f", recall)
sprintf("Caret f1_score: %.15f", f1)

mcc <- (tp * tn - fp * fn) / sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
print(paste("MCC (manual):", mcc))

kappa_caret <- confMatrix$overall['Kappa']
print(paste("Cohen's Kappa (caret):", kappa_caret))

sensitivity1 <- tp / (tp + fn)
specificity1 <- tn / (tn + fp)

balanced_accuracy1 = (sensitivity1 + specificity1)/2
sprintf("Balanced Accuracy (caret): %.15f", balanced_accuracy1)


beta <- 0.5;
f_beta_score <- ((1 + beta**2) * (precision * recall)) / (beta**2 * precision + recall);
sprintf("Balanced Accuracy (caret): %.15f", f_beta_score)


jaccard <- tp/ (tp + fp + fn)
sprintf("Jaccard Index (caret): %.15f", jaccard)

sensitivity <- confMatrix$byClass["Sensitivity"]
specificity <- confMatrix$byClass["Specificity"]
geometric_mean_caret <- sqrt(sensitivity * specificity)
print(paste("Geometric Mean (caret):", geometric_mean_caret))


[1] "Caret Accuracy: 0.691011235955056"

[1] "Caret precision: 0.775641025641026"

[1] "Caret recall: 0.858156028368794"

[1] "Caret f1_score: 0.814814814814815"

[1] "MCC (manual): -0.108238414016216"
[1] "Cohen's Kappa (caret): -0.103222898354744"


[1] "Balanced Accuracy (caret): 0.456105041211424"

[1] "Balanced Accuracy (caret): 0.790849673202615"

[1] "Jaccard Index (caret): 0.035087719298246"

[1] "Geometric Mean (caret): 0.215375979032619"


In [6]:
### MLmetrics Library ###
accuracy <- Accuracy(y_pred, y_true)
sprintf("MLmetrics Accuracy: %.15f", accuracy)

precision1 <- Precision(y_true, y_pred, positive = '1')
precision2 <- Precision(y_true, y_pred, positive = '0')
#precision <- (precision2 + precision1)/2
sprintf("MLmetric Precision: %.15f", precision1)

recall1 <- Recall(y_true, y_pred, positive = "1")
recall2 <- Recall (y_true, y_pred, positive = "0") 
sprintf("MLmetric Recall: %.15f", recall1)
f1_score1 <- F1_Score(y_pred, y_true, positive = "1")
f1_score2 <- F1_Score(y_pred, y_true, positive = "0")

sprintf("MLmetric F1: %.15f", f1_score1)

[1] "MLmetrics Accuracy: 0.691011235955056"

[1] "MLmetric Precision: 0.090909090909091"

[1] "MLmetric Recall: 0.054054054054054"

[1] "MLmetric F1: 0.067796610169492"

In [7]:
### MCCR Library ###
mcc <- mccr(y_true, y_pred)
sprintf("MccR MCC: %.15f", mcc)

[1] "MccR MCC: -0.108238414016216"

In [8]:
data1 <- tibble(
  truth = factor(y_true),
  prediction = factor(y_pred)
)

# Calculate Accuracy
accuracy_result <- accuracy_vec(truth = data1$truth, estimate = data1$prediction)
print(sprintf("YardStick Accuracy: %.15f", accuracy_result))

# Calculate Precision
precision_yard <- precision_vec(truth = data1$truth, estimate = data1$prediction)
print(sprintf("YardStick Precision: %.15f", precision_yard))

# Calculate Recall
recall_yard <- recall_vec(truth = data1$truth, estimate = data1$prediction)
print(sprintf("YardStick Recall: %.15f", recall_yard))

# Calculate F1 Score
f1_score_result <- f_meas_vec(truth = data1$truth, estimate = data1$prediction)
print(sprintf("F1 Score (yardstick): %.15f", f1_score_result))

# Calculate Kappa
kappa_yardstick <- kap_vec(truth = data1$truth, estimate = data1$prediction)
print(sprintf("YardStick Kappa: %.15f", kappa_yardstick))

# Calculate F-beta Score with beta=0.5
f_beta_score_result <- f_meas_vec(truth = data1$truth, estimate = data1$prediction, beta = 0.5)
print(sprintf("YardStick FbetaScore: %.15f", f_beta_score_result))

# Calculate Sensitivity and Specificity
sensitivity_metric <- sens_vec(truth = data1$truth, estimate = data1$prediction)
specificity_metric <- spec_vec(truth = data1$truth, estimate = data1$prediction)
geometric_mean_yardstick <- sqrt(sensitivity_metric * specificity_metric)
print(sprintf("Geometric Mean (yardstick): %.15f", geometric_mean_yardstick))


[1] "YardStick Accuracy: 0.691011235955056"
[1] "YardStick Precision: 0.775641025641026"
[1] "YardStick Recall: 0.858156028368794"
[1] "F1 Score (yardstick): 0.814814814814815"
[1] "YardStick Kappa: -0.103222898354744"
[1] "YardStick FbetaScore: 0.790849673202614"
[1] "Geometric Mean (yardstick): 0.215375979032619"


In [9]:
### Psych Library ###
kappa_psych <- psych::cohen.kappa(cbind(factor(y_true, levels = c(0, 1)), factor(y_pred, levels = c(0, 1))))
print(paste("Cohen's Kappa (psych):", kappa_psych$kappa))

[1] "Cohen's Kappa (psych): -0.103222898354744"


In [10]:
### Irr Library ###
kappa_irr <- irr::kappa2(cbind(factor(y_true, levels = c(0, 1)), factor(y_pred, levels = c(0, 1))))
print(paste("Cohen's Kappa (irr):", kappa_irr$value))

[1] "Cohen's Kappa (irr): -0.103222898354744"
